# IF3170 Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [ ]:
# Download Library
%pip install pandas
%pip install openpyxl
%pip install scikit-learn
%pip install joblib

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from joblib import Parallel, delayed
from sklearn.impute import SimpleImputer

# Import other libraries if needed

## Import Dataset

In [ ]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

df = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')
df.head()

Train Dataset

In [ ]:
#https://drive.google.com/file/d/1qTk4wuXE8NjOsqoEERFre9d9UwWtfOlf/view?usp=sharing
#https://drive.google.com/file/d/1lQnlxexMO5po0TFcOk5dp-tnkR_tswr9/view?usp=sharing
#https://drive.google.com/file/d/1S9niayLbd1AGFqSr7IK7oroKyQalFN63/view?usp=sharing
#https://drive.google.com/file/d/1W2EossQ9NWgzySeUFRqetvhOAKgpusnk/view?usp=sharing
#https://drive.google.com/file/d/18IzeH8Xn4MIRS9MKXJkTdEvif1GRStuw/view?usp=sharing
#https://drive.google.com/file/d/1-qmbqLwPzv3v2gdY4cdumGKV6J7q7P2w/view?usp=sharing

# time_features_train = pd.read_csv('https://drive.google.com/uc?id=1qTk4wuXE8NjOsqoEERFre9d9UwWtfOlf')
# labels_train = pd.read_csv('https://drive.google.com/uc?id=1lQnlxexMO5po0TFcOk5dp-tnkR_tswr9')
# flow_features_train = pd.read_csv('https://drive.google.com/uc?id=1S9niayLbd1AGFqSr7IK7oroKyQalFN63')
# content_features_train = pd.read_csv('https://drive.google.com/uc?id=1W2EossQ9NWgzySeUFRqetvhOAKgpusnk')
# basic_features_train = pd.read_csv('https://drive.google.com/uc?id=18IzeH8Xn4MIRS9MKXJkTdEvif1GRStuw')
# additional_features_train = pd.read_csv('https://drive.google.com/uc?id=1-qmbqLwPzv3v2gdY4cdumGKV6J7q7P2w')               

time_features_train = pd.read_csv("train/time_features_train.csv")  
labels_train = pd.read_csv('train/labels_train.csv')
flow_features_train = pd.read_csv('train/flow_features_train.csv')
content_features_train = pd.read_csv('train/content_features_train.csv')
basic_features_train = pd.read_csv('train/basic_features_train.csv')
additional_features_train = pd.read_csv('train/additional_features_train.csv')


In [ ]:
print(time_features_train.head())
print(labels_train.head())
print(flow_features_train.head())
print(content_features_train.head())
print(basic_features_train.head())
print(additional_features_train.head())

Test Dataset

In [ ]:
#https://drive.google.com/file/d/1Atr6sA2Tzc0D9L_bQM2_6KLYh332Iv0y/view?usp=sharing
#https://drive.google.com/file/d/1ePaWIPvDWCpEzrTChPy15t_F4BgotXmD/view?usp=sharing
#https://drive.google.com/file/d/1-EA1sY0DBnHcnQ0LOSZ_eVR6C0GC2eEd/view?usp=sharing
#https://drive.google.com/file/d/1Dc14IM0m16HRDXjy8aQ9JCAIfQhkPPvi/view?usp=sharing
#https://drive.google.com/file/d/1Y8hyL-xSee0KFwuXo9RORo430DTQ4bV7/view?usp=sharing


# time_features_test = pd.read_csv('https://drive.google.com/uc?id=1Atr6sA2Tzc0D9L_bQM2_6KLYh332Iv0y')
# flow_features_test = pd.read_csv('https://drive.google.com/uc?id=1ePaWIPvDWCpEzrTChPy15t_F4BgotXmD')
# content_features_test = pd.read_csv('https://drive.google.com/uc?id=1-EA1sY0DBnHcnQ0LOSZ_eVR6C0GC2eEd')
# basic_features_test = pd.read_csv('https://drive.google.com/uc?id=1Dc14IM0m16HRDXjy8aQ9JCAIfQhkPPvi')
# aditional_features_test = pd.read_csv('https://drive.google.com/uc?id=1Y8hyL-xSee0KFwuXo9RORo430DTQ4bV7')

time_features_test = pd.read_csv("test/time_features_test.csv")
flow_features_test = pd.read_csv('test/flow_features_test.csv')
content_features_test = pd.read_csv('test/content_features_test.csv')
basic_features_test = pd.read_csv('test/basic_features_test.csv')
aditional_features_test = pd.read_csv('test/additional_features_test.csv')

In [ ]:
print(time_features_test.head())
print(flow_features_test.head())
print(content_features_test.head())
print(basic_features_test.head())
print(aditional_features_test.head())

In [ ]:
merged_train_df = pd.merge(time_features_train, labels_train, on='id', how='outer')
merged_train_df = pd.merge(merged_train_df, flow_features_train, on='id', how='outer')
merged_train_df = pd.merge(merged_train_df, content_features_train, on='id', how='outer')
merged_train_df = pd.merge(merged_train_df, basic_features_train, on='id', how='outer')
merged_train_df = pd.merge(merged_train_df, additional_features_train, on='id', how='outer')

# merged_train_df.to_csv('merged_train_dataset.csv', index=False)
# merged_train_df.to_excel('merged_train_dataset.xlsx', index=False)

In [ ]:
merged_test_df = pd.merge(flow_features_test, time_features_test, on='id', how='outer')
merged_test_df = pd.merge(merged_test_df, content_features_test, on='id', how='outer')
merged_test_df = pd.merge(merged_test_df, basic_features_test, on='id', how='outer')
merged_test_df = pd.merge(merged_test_df, aditional_features_test, on='id', how='outer')

# merged_test_df.to_csv('merged_test_dataset.csv', index=False)
# merged_test_df.to_excel('merged_test_dataset.xlsx', index=False)

# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [ ]:
def explore_data(data, dataset_name):
    print(f"Eksplorasi dataset {dataset_name}")
    print("="*50)
    
    print(f"Shape dataset: {data.shape}\n")
    
    print("Tipe-tipe feature")
    print(data.dtypes.value_counts(), "\n")
    
    print("Missing Values (Jumlah & Persentase):")
    missing_values = data.isnull().sum()
    missing_percentage = (missing_values / len(data)) * 100
    missing_summary = pd.DataFrame({
        "Missing Values": missing_values,
        "Persentase": missing_percentage
    })

    print(missing_summary[missing_summary["Missing Values"] > 0].sort_values(by="Persentase", ascending=False), "\n")
    
    print("Informasi Statistik:")
    print(data.describe().T, "\n")
    
    print("Nilai feature kategorikal: ")
    categorical_cols = data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        print(f"{col}:")
        print(data[col].value_counts(), "\n")
    
    if "attack_cat" in data.columns:
        print("Distribusi variabel target: ")
        print(data["attack_cat"].value_counts(), "\n")

In [ ]:
explore_data(merged_train_df, "Train")
explore_data(merged_test_df, "Test")

In [ ]:
for column in merged_train_df.columns:
    print(f"{column}, Tipe Data: {merged_train_df[column].dtype}")

In [ ]:
for column in merged_train_df.columns:
    if merged_train_df[column].dtype == 'object': 
        unique_values = merged_train_df[column].unique()
        print(f"Kolom : {column}, Unique Values: {unique_values}")

In [ ]:
def analyze_distribution(data, features):
    for feature in features:
        if data[feature].isnull().sum() > 0:  
            print(f"Feature: {feature}")
            print(data[feature].describe())
            skewness = data[feature].skew()
            print(f"Skewness: {skewness}")
            if skewness < -1 or skewness > 1:
                print("Skewed. Imputasi pake median")
            elif -1 <= skewness <= 1:
                print("Lumayan Simetris. Imputasi pake nilai mean")
            print("-" * 50)

print("Distribusi Fitur Numerik:")
analyze_distribution(merged_train_df, merged_train_df.select_dtypes(include=['float64', 'int64']).columns)

print("Distribusi Fitur Kategorikal")
for feature in merged_train_df.select_dtypes(include=['object']).columns:
    if merged_train_df[feature].isnull().sum() > 0:
        print(f"Feature: {feature}")
        print(merged_train_df[feature].value_counts())
        print(f"Mode: {merged_train_df[feature].mode()[0]}")
        print("-" * 50)

In [ ]:
def detect_outliers(df, features):
    outlier_summary = {}
    for feature in features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
        outlier_percentage = len(outliers) / len(df) * 100
        
        outlier_summary[feature] = {
            'Total outlier': len(outliers),
            'Persentase outlier': outlier_percentage,
            'batas bawah': lower_bound,
            'batas atas': upper_bound
        }
    
    return pd.DataFrame.from_dict(outlier_summary, orient='index')

columns_to_check = [col for col in merged_train_df.select_dtypes(include=['float64', 'int64']).columns if col != 'id']
outliers = detect_outliers(merged_train_df, columns_to_check)

outliers

In [ ]:
print(merged_train_df.columns)

In [ ]:
def compare_outlier_methods(df, feature):
    original_skewness = df[feature].skew()
    
    # 1. Imputation (Median)
    imputed_median = df[feature].fillna(df[feature].median())
    imputed_median_skewness = imputed_median.skew()
    
    # 2. Clipping
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    clipped = df[feature].clip(lower=lower_bound, upper=upper_bound)
    clipped_skewness = clipped.skew()
    
    # 3. Log Transformation
    log_transformed = np.log1p(df[feature])
    log_skewness = log_transformed.skew()
    
    # 4. Square Root Transformation
    sqrt_transformed = np.sqrt(df[feature])
    sqrt_skewness = sqrt_transformed.skew()
    
    return {
        'original_skewness': original_skewness,
        'imputed_median_skewness': imputed_median_skewness,
        'clipped_skewness': clipped_skewness,
        'log_skewness': log_skewness,
        'sqrt_skewness': sqrt_skewness
    }

results = {}

merged_train_df_check = merged_train_df.copy()

for feature in columns_to_check:
    results[feature] = compare_outlier_methods(merged_train_df_check, feature)

pd.DataFrame.from_dict(results, orient='index')


In [ ]:
def find_best_outlier_method(results):
    best_methods = {}
    
    for feature, skewness_values in results.items():
        best_method = min(skewness_values, key=lambda method: abs(skewness_values[method]))
        best_methods[feature] = best_method
    
    return best_methods

best_methods = find_best_outlier_method(results)

for feature, method in best_methods.items():
    print(f"Fitur '{feature}' -> teknik : {method}")

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [ ]:
id = merged_train_df['id']
X = merged_train_df.drop(columns=['id','label', 'attack_cat'], errors='ignore')
y = merged_train_df['attack_cat']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

train_val_split_info = {
    "X_train_shape": X_train.shape,
    "y_train_shape": y_train.shape,
    "X_val_shape": X_val.shape,
    "y_val_shape": y_val.shape
}

train_val_split_info

In [ ]:
print(X_train.columns)
print(X_val.columns)

In [ ]:
print(X_train.isnull().sum())
print(X_val.isnull().sum())

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:
def handle_missing_values(merged_train_df):
    df = merged_train_df.copy()
    
    highly_skewed_numeric = [
        'sjit', 'djit', 'sinpkt', 'dinpkt', 'tcprtt', 'synack', 'ackdat', 
        'stcpb', 'dtcpb', 'smean', 'dmean', 'trans_depth', 'response_body_len', 
        'dur', 'sbytes', 'dbytes', 'sloss', 'dloss', 'sload', 'dload', 
        'spkts', 'dpkts', 'is_sm_ips_ports', 'ct_flw_http_mthd', 
        'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 
        'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 
        'ct_dst_src_ltm'
    ]
    
    symmetric_numeric = [
        'swin', 'dwin', 'sttl', 'dttl', 'ct_state_ttl'
    ]
    
    categorical_columns = [
        'proto', 'state', 'service'
    ]
    
    median_imputer = SimpleImputer(strategy='median')
    df[highly_skewed_numeric] = median_imputer.fit_transform(df[highly_skewed_numeric])

    mean_imputer = SimpleImputer(strategy='mean')
    df[symmetric_numeric] = mean_imputer.fit_transform(df[symmetric_numeric])
    
    mode_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_columns] = mode_imputer.fit_transform(df[categorical_columns])
    
    return df

X_train_transformed = handle_missing_values(X_train)
X_val_transformed = handle_missing_values(X_val)

print("Cek missing values:")
print(X_train_transformed.isnull().sum())
print(X_val_transformed.isnull().sum())

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
def process_and_transform_feature(X, feature, original_df):
    original_skewness = X[feature].skew()
    
    # 1. Imputation (Median)
    imputed_median = X[feature].fillna(original_df[feature].median())
    imputed_median_skewness = imputed_median.skew()
    
    # 2. Clipping
    Q1 = original_df[feature].quantile(0.25)
    Q3 = original_df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    clipped = X[feature].clip(lower=lower_bound, upper=upper_bound)
    clipped_skewness = clipped.skew()
    
    # 3. Log Transformation
    log_transformed = np.log1p(X[feature])
    log_skewness = log_transformed.skew()
    
    # 4. Square Root Transformation
    sqrt_transformed = np.sqrt(X[feature])
    sqrt_skewness = sqrt_transformed.skew()
    
    methods = {
        'original_skewness': original_skewness,
        'imputed_median_skewness': imputed_median_skewness,
        'clipped_skewness': clipped_skewness,
        'log_skewness': log_skewness,
        'sqrt_skewness': sqrt_skewness
    }

    best_method = min(methods, key=lambda k: abs(methods[k]))

    if best_method == 'original_skewness':
        transformed_feature = X[feature]
    elif best_method == 'imputed_median_skewness':
        transformed_feature = imputed_median
    elif best_method == 'clipped_skewness':
        transformed_feature = clipped
    elif best_method == 'log_skewness':
        transformed_feature = log_transformed
    elif best_method == 'sqrt_skewness':
        transformed_feature = sqrt_transformed
    
    return transformed_feature, best_method

def transform_dataframe(X, features, original_df):
    transformed_X = X.copy()
    best_methods = {}
    
    for feature in features:
        if feature in X.columns:
            transformed_feature, best_method = process_and_transform_feature(X, feature, original_df)
            transformed_X[feature] = transformed_feature
            best_methods[feature] = best_method
    
    return transformed_X, best_methods

columns_to_check = [col for col in X_train.select_dtypes(include=['float64', 'int64']).columns if col != 'id']

X_train_transformed, train_best_methods = transform_dataframe(X_train_transformed, columns_to_check, X_train)
X_val_transformed, val_best_methods = transform_dataframe(X_val_transformed, columns_to_check, X_val)

In [ ]:
def check_skewness_after_transformation(df, columns_to_check):
    skewness_results = {}
    for feature in columns_to_check:
        skewness_results[feature] = df[feature].skew()
        print(f"Feature: {feature} :  {skewness_results[feature]:.6f}")
    return skewness_results

print(check_skewness_after_transformation(X_train_transformed, columns_to_check))
print(check_skewness_after_transformation(X_val_transformed, columns_to_check))


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
# Write your code here

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
# Write your code here

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
class FeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.label_encoder = None
        self.one_hot_encoder = None
        self.one_hot_columns = ['proto', 'state', 'service']
        self.label_column = 'attack_cat'

    def fit(self, X, y=None):
        if self.label_column in X.columns:
            self.label_encoder = LabelEncoder()
            X[self.label_column] = X[self.label_column].fillna('Unknown')
            self.label_encoder.fit(X[self.label_column])

        existing_one_hot_columns = [col for col in self.one_hot_columns if col in X.columns]
        if existing_one_hot_columns:
            self.one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            self.one_hot_encoder.fit(X[existing_one_hot_columns])

        return self

    def transform(self, X):
        X_transformed = X.copy()

        if self.label_column in X_transformed.columns:
            X_transformed[self.label_column] = X_transformed[self.label_column].fillna('Unknown')
            X_transformed[self.label_column] = self.label_encoder.transform(X_transformed[self.label_column])

        existing_one_hot_columns = [col for col in self.one_hot_columns if col in X_transformed.columns]
        if existing_one_hot_columns:
            encoded_features = self.one_hot_encoder.transform(X_transformed[existing_one_hot_columns])
            encoded_df = pd.DataFrame(
                encoded_features,
                columns=self.one_hot_encoder.get_feature_names_out(existing_one_hot_columns),
                index=X_transformed.index
            )

            X_transformed = X_transformed.drop(columns=existing_one_hot_columns)
            X_transformed = pd.concat([X_transformed, encoded_df], axis=1)

        return X_transformed

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
# Write your code here

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [ ]:
# Write your code here

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [ ]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)


feature_encoder_pipeline = Pipeline([
    ('encoder', FeatureEncoder())
])

In [ ]:
X_train_transformed = feature_encoder_pipeline.fit_transform(X_train)
X_val_transformed = feature_encoder_pipeline.transform(X_val)
y_val_encoded = label_encoder.transform(y_val)
y_train_encoded = label_encoder.transform(y_train)
merged_test_df_transformed = feature_encoder_pipeline.transform(merged_test_df)

or create your own here

In [ ]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [ ]:
X_train_downsampled, y_train_downsampled = resample(
    X_train_transformed, y_train_encoded, 
    replace=False,  
    n_samples=int(0.0001 * len(X_train)),  
    random_state=42
)

In [ ]:
class KNNScratch:
    def __init__(self, k_neighbours=3, distance_type='euclidean', p=3):
        self.k_neighbours = k_neighbours
        self.distance_type = distance_type
        self.p = p
        print(f"[DEBUG] KNN initialized with k_neighbours={k_neighbours}, distance_type={distance_type}, p={p}")
        
    def fit(self, X_train, y_train):
        self.X_train = X_train.to_numpy() if isinstance(X_train, pd.DataFrame) else X_train
        self.y_train = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train

        if not isinstance(self.X_train, np.ndarray):
            raise ValueError("[DEBUG] X_train is not a numpy array.")
        if not isinstance(self.y_train, np.ndarray):
            raise ValueError("[DEBUG] y_train is not a numpy array.")

        print(f"[DEBUG] Model fitted with {len(self.X_train)} training samples.")
        print(f"[DEBUG] X_train shape: {self.X_train.shape}")
        print(f"[DEBUG] y_train shape: {self.y_train.shape}")
        print(f"[DEBUG] Sample X_train[0]: {self.X_train[0]}")
        print(f"[DEBUG] Sample y_train[0]: {self.y_train[0]}")
        
    def calculate_distance(self, x1, x2):
        try:
            print(f"[DEBUG] Calculating distance between x1 and x2")
            print(f"[DEBUG] x1 shape: {x1.shape}, x2 shape: {x2.shape}")
            if self.distance_type == 'euclidean':
                distance = np.sqrt(np.sum((x1 - x2) ** 2))
            elif self.distance_type == 'manhattan':
                distance = np.sum(np.abs(x1 - x2))
            elif self.distance_type == 'minkowski':
                distance = np.sum(np.abs(x1 - x2) ** self.p) ** (1 / self.p)
            else:
                raise ValueError("Tipe tidak diketahui: pilih 'euclidean', 'manhattan', atau 'minkowski'")
            return distance
        except Exception as e:
            print(f"[DEBUG] Error in calculate_distance: x1={x1}, x2={x2}, Error={e}")
            raise
        
    def predict(self, X_test):
        X_test = X_test.to_numpy() if isinstance(X_test, pd.DataFrame) else X_test

        print(f"[DEBUG] Predicting {len(X_test)} samples.")
        print(f"[DEBUG] X_test shape: {X_test.shape}")
        print(f"[DEBUG] Sample X_test[0]: {X_test[0]}")

        predictions = []
        for idx, x in enumerate(X_test):
            try:
                print(f"[DEBUG] Predicting sample {idx}")
                result = self.predict_one(x)
                predictions.append(result)
            except Exception as e:
                print(f"[DEBUG] Error at index {idx}: {e}")
                print(f"[DEBUG] Problematic sample: {x}")
                raise
        return np.array(predictions)
    
    def predict_one(self, x):
        print(f"[DEBUG] Predicting for one sample.")
        print(f"[DEBUG] Current sample x: {x}")
        
        # Jarak antara x dan semua titik train
        distances = []
        for idx, x_train in enumerate(self.X_train):
            try:
                distance = self.calculate_distance(x, x_train)
                distances.append(distance)
            except Exception as e:
                print(f"[DEBUG] Error in distance calculation at index {idx}: x={x}, x_train={x_train}, Error={e}")
                raise
                
        print(f"[DEBUG] Distances for one test sample calculated.")
        
        # Indeks k tetangga terdekat
        k_indices = np.argsort(distances)[:self.k_neighbours]
        print(f"[DEBUG] k_indices: {k_indices}")
        
        # Label dari k tetangga terdekat
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        print(f"[DEBUG] k_nearest_labels: {k_nearest_labels}")
        
        # Label yang paling umum di antara k tetangga terdekat
        most_common_label = max(set(k_nearest_labels), key=k_nearest_labels.count)
        print(f"[DEBUG] Most common label: {most_common_label}")
        
        return most_common_label


In [ ]:
X_test = merged_test_df_transformed = merged_test_df_transformed.astype(float)

k_neighbors = 1  
knn_model = KNNScratch(k_neighbours=k_neighbors, distance_type='euclidean')
knn_model.fit(X_train_downsampled, y_train_downsampled)

y_test_predictions_encoded = knn_model.predict(X_test)

In [ ]:
def validate_numeric_data(df):

    non_numeric_columns = []
    for column in df.columns:
        non_numeric_values = df[column][~df[column].apply(lambda x: isinstance(x, (int, float)))]
        if not non_numeric_values.empty:
            non_numeric_columns.append(column)
            print(f"[DEBUG] Non-numeric values found in column '{column}':")
            print(non_numeric_values.head())
    
    if non_numeric_columns:
        print("[DEBUG] The following columns have non-numeric values and will be converted to NaN:")
        print(non_numeric_columns)
        df = df.apply(pd.to_numeric, errors='coerce')
    else:
        print("[DEBUG] All values in the dataframe are numeric.")
    
    return df

X_test_cleaned = validate_numeric_data(merged_test_df_transformed)

print("[DEBUG] Data types after cleaning:")
print(X_test_cleaned.dtypes)

X_test = X_test_cleaned.astype(float)
print("[DEBUG] Sample data after cleaning and conversion to float:")
print(X_test.head())

In [ ]:
# X_test = merged_test_df_transformed = merged_test_df_transformed.astype(float)

# k_neighbors = 1  
# knn_model = KNNScratch(k_neighbours=k_neighbors, distance_type='euclidean')
# knn_model.fit(X_train_downsampled, y_train_downsampled)

# def predict_batch(x):
#     return knn_model.predict_one(x)

# predictions = Parallel(n_jobs=-1)(delayed(predict_batch)(x) for x in X_test)

In [ ]:
# test_ids = merged_test_df['id']

# y_test_predictions = label_encoder.inverse_transform(predictions)

# forecast_output = pd.DataFrame({
#     'id': test_ids,
#     'attack_cat': y_test_predictions
# })

# forecast_output_path = "forecast_output.csv"
# forecast_output.to_csv(forecast_output_path, index=False)

## B. Naive Bayes

In [ ]:
# Type your code here

## C. ID3

In [ ]:
# Type your code here

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`